If in VSCODE:

In [1]:
import torch
print(torch.__version__)
torch.cuda.is_available()


2.1.0


True

Hyper params

In [1]:
# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# Fine-tuned model name
new_model = "StarkWizard/llama-2-7b-cairo-trained-PEFT"



import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "StarkWizard/codellama-cairo-instruct"

max_steps = 700 # to tweak to get the best out of the model 

running this cell is mandatory

Push Model To Hub

- Make sure to reinit the kernel
- We reload the base model
- Load the Peft adapter
- Push the merged model

NOTE:

We use CPU because of the error: Cannot merge LORA layers when the model is loaded in 8-bit mode
Merging is fast, so it's not an issue

In [2]:

from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
from transformers import  AutoTokenizer, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name,
                                             trust_remote_code=True,
                                             low_cpu_mem_usage=True,
                                             device_map={"": "cpu"},
                                             torch_dtype=torch.float16
                                             )

model_to_merge  = PeftModel.from_pretrained(model, new_model,
                        torch_dtype=torch.float16, 
                        device_map={"": "cpu"}
                         )
merged_model = model_to_merge.merge_and_unload()
#model.save_pretrained("cairo-mistral")
merged_model.push_to_hub(hub_name,max_shard_size="1GB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00013-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00011-of-00014.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

pytorch_model-00010-of-00014.bin:   0%|          | 0.00/944M [00:00<?, ?B/s]

pytorch_model-00012-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

pytorch_model-00004-of-00014.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

pytorch_model-00003-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00002-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00001-of-00014.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/StarkWizard/codellama-cairo-instruct/commit/a2b83683be105cd5bd3cec953fdd5f76eaad08fb', commit_message='Upload LlamaForCausalLM', commit_description='', oid='a2b83683be105cd5bd3cec953fdd5f76eaad08fb', pr_url=None, pr_revision=None, pr_num=None)